**By using the output of this notebook, you are accepting the [competition rules](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/rules).**


**This is originally from https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-png-256x256 by @xhlulu, I just modified a bit to save metadata for test images**

## References

- Monochrome fix and scaling: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
- Resizing and saving image: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
image_id = []
dim0 = []
dim1 = []

#for split in ['train', 'test']:
for split in ['test']:
    load_dir = f'../input/vinbigdata-chest-xray-abnormalities-detection/{split}/'
    save_dir = f'/kaggle/tmp/{split}/'

    os.makedirs(save_dir, exist_ok=True)

    for file in tqdm(os.listdir(load_dir)):
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(load_dir + file)
        #im = resize(xray, size=256)  
        #im.save(save_dir + file.replace('dicom', 'png'))
        
        image_id.append(file.replace('.dicom', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])

In [ ]:
file_list = os.listdir(load_dir)

In [ ]:
from joblib import Parallel, delayed

def load_meta(load_dir: str, file: str):
    xray = read_xray(load_dir + file, False, False)
    image_id = file.replace('.dicom', '')
    height, width = xray.shape[:2]
    return image_id, height, width

print(f"total {len(file_list)}")
n_jobs = 16
results = Parallel(n_jobs, verbose=1)(
    delayed(load_meta)(load_dir, filename) for filename in file_list)

In [ ]:
test_meta = pd.DataFrame(results, columns=["image_id", "dim0", "dim1"])
test_meta.to_csv("test_meta.csv", index=False)